In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, Bidirectional, TimeDistributed
from tensorflow.keras.layers import MaxPooling1D, Flatten
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.metrics import RootMeanSquaredError

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/output_data.csv', newline='') as f:
    reader = csv.reader(f, delimiter=';')
    i=0
    for row in reader:
        i+=1
    length_of_data=i+1
with open('/content/drive/MyDrive/Colab Notebooks/output_data.csv', newline='') as f:
    reader = csv.reader(f, delimiter=';')
    #x_train=np.zeros((length_of_data-1,140),np.float32)
    i=0
    row1=[None]*100
    x_train=[]
    for row in reader:
        j=40
        while j <len(row):
            row1[j-40]=np.float32(row[j])
            j+=1
        x_train.append(row1.copy())
        i+=1
with open('/content/drive/MyDrive/Colab Notebooks/output_labels_forecasting.csv', newline='') as f:
    reader = csv.reader(f, delimiter=';')
    #y_train=np.zeros((length_of_data-1,3),np.float32)
    i=0
    j=0
    y_train=[]
    row1=[None]*3
    for row in reader:
        j=0
        for j in range(3):
            row1[j]=np.float32(row[j])
        y_train.append(row1.copy())
        i+=1
print('shape x:',np.shape(x_train),'shape y:', np.shape(y_train))

shape x: (126957, 100) shape y: (126917, 3)


In [ ]:
fo_fur=[]
for row in x_train:
  fo_fur.append(row[0].copy())

fft = tf.signal.rfft(fo_fur)
f_per_dataset = np.arange(0, len(fft))

n_samples_h = len(fo_fur)
minutes_per_year = 60*24*365.2524
years_per_dataset = n_samples_h/(minutes_per_year)

f_per_year = f_per_dataset/years_per_dataset
plt.step(f_per_year, np.abs(fft))
plt.xscale('log')
plt.ylim(0, 5)
plt.xlim([0.1, max(plt.xlim())])
plt.xticks([1, 365.2524], labels=['1/Year', '1/day'])
_ = plt.xlabel('Frequency (log scale)')


In [ ]:
print(fo_fur[0:10])

[1.0414605, 1.0415388, 1.0428774, 1.03419, 1.0370721, 1.0346698, 1.0354108, 1.0381526, 1.037037, 1.0386205]


In [ ]:
np.reshape(x_train, (np.shape(x_train)[0],np.shape(x_train)[1],1))
np.reshape(y_train, (np.shape(y_train)[0],np.shape(y_train)[1],1))
print('shape x:',np.shape(x_train),'shape y:', np.shape(y_train))

shape x: (126957, 140) shape y: (126957, 2)


In [ ]:
batch_size = 16
print(np.shape(x_train))
# Reserve 1000 samples for validation.
x_val = x_train[-3040:-1040][:]
y_val = y_train[-3000:-1000][:]
x_test=x_train[-1040:-40][:]
y_test=y_train[-1000:][:]
x_train1 = x_train[:50000][:]
y_train1 = y_train[:50000][:]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train1, y_train1))
train_dataset = train_dataset.batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(batch_size)

(126957, 100)


In [ ]:
buff1=0
for i in range(20000):
  buff=0
  for j in range (3):
    buff=buff+abs(y_train1[i][j]-1.0)
  buff1=buff1+buff/3
print(buff1/20000)

0.000813898895184197


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Reshape((100, 1)))
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(None, 100, 1)))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())

# LSTM layers
model.add(tf.keras.layers.Reshape((640, 1)))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(100, return_sequences=False)))
model.add(Dropout(0.5))

#Final layers
model.add(Dense(3,use_bias=True,bias_initializer=keras.initializers.Constant(1.), activation='linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Reshape((100, 1)),
  tf.keras.layers.Conv1D(
    32,
    [7],
    strides=1,
    padding="valid",
    activation='relu',
    use_bias=True,
    bias_initializer="zeros",
    ),
  tf.keras.layers.Conv1D(
    16,
    [7],
    strides=1,
    padding="valid",
    activation='relu',
    use_bias=True,
    bias_initializer="zeros",
    ),
  tf.keras.layers.MaxPooling1D(
    pool_size=3, padding="valid",
  ),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(3,use_bias=True,bias_initializer=keras.initializers.Constant(1.),activation=tf.nn.relu)
])
#0.35

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Reshape((140, 1)),
  tf.keras.layers.LSTM(units=70, return_sequences=True),
  tf.keras.layers.LSTM(units=20),
  #tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(80, activation='relu'),
  tf.keras.layers.Dense(40, use_bias=True, bias_initializer=keras.initializers.Constant(1.), activation='linear')
])

In [ ]:
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.01),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryCrossentropy(), tf.keras.metrics.MeanAbsoluteError()],
)

In [ ]:
model.fit(train_dataset, epochs=2)

Epoch 1/2
3125/3125 [==============================] - 22s 3ms/step - loss: -1.6328e-04 - binary_crossentropy: -1.6328e-04 - mean_absolute_error: 0.2020
Epoch 2/2
3125/3125 [==============================] - 10s 3ms/step - loss: -3.0444e-04 - binary_crossentropy: -3.0444e-04 - mean_absolute_error: 0.2023


In [ ]:
print("Predict")
#predictions = model.predict(test_dataset)
print('',file=open('/content/drive/MyDrive/Colab Notebooks/result_last_1000.txt','w'))
i=0
for i in range(1000):
    if i>100:
        break
    print(model(np.array(x_test[i:i+1])),'true:',y_test[i:i+1],file=open('/content/drive/MyDrive/Colab Notebooks/result_last_1000.txt','a'))

Predict


In [ ]:
print("Evaluate")
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

Evaluate
63/63 [==============================] - 0s 3ms/step - loss: -4.1816e-04 - binary_crossentropy: -4.1816e-04 - mean_absolute_error: 0.2023


{'binary_crossentropy': -0.0004181632830295712,
 'loss': -0.0004181632830295712,
 'mean_absolute_error': 0.20225484669208527}

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/lstm_forecasting1")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/lstm_forecasting1/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/lstm_forecasting1/assets
